In [1]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Module to see files and folders in directories
from scipy import stats
from scipy import optimize

In [2]:
# Defining the parameters:
# mu1是L_top，mu2是L_bottom
mu1   =  2.0029
sig1  =  0.0003
mu2   =  2.835
sig2  =  0.03
rho12 =  0         # Correlation parameter!

In [3]:
if not (-1.0 <= rho12 <= 1.0): 
    raise ValueError(f"Correlation factor not in interval [-1,1], as it is {rho12:6.2f}")

In [4]:
from IPython.core.display import Latex

def lprint(*args,**kwargs):
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)

def myDiff(formula):
    return sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2)

def myDiffWithCorr(formula, name = "", printNow = False):
    dd = sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2 + 2*(formula.diff(L)*formula.diff(W)*(sigCorr**2)))
    if(printNow):
        lprint(latex(Eq(symbols('sigma_'+name), dd)))
    fd = lambdify((L,dL,W,dW,sigCorr),dd)
    return dd, fd
    
def diff_and_print(formula, name = ""):
    # Calculate uncertainty and print original relation/formula and the uncertainty
    dd = myDiff(formula)
    lprint(latex(Eq(symbols(name),formula)))
    lprint(latex(Eq(symbols('sigma_'+name), dd)))
    
def lambdifyFormula(formula, *args, name = ""):
    # Turn expression into numerical functions 
    f = lambdify((L,W),formula)
    d = myDiff(formula)
    fd = lambdify((L,dL,W,dW),d)
    return f, fd

In [5]:
# Import SymPy: 
from sympy import * 
    
# Define variables:
L,W,D = symbols("L, W, D")
dL,dW,dD = symbols("sigma_L, sigma_W, sigma_D")

# Define relations:
# Diagonal
# D = L * (((2*np.pi)/W)**2) g的公式
D = L * (((2*np.pi)/W)**2)

# Try writing a simple function to not repeat yourself! (See cell above)
diff_and_print(D,"D")

dD = myDiff(D)

# Turn expressions into numerical functions 
fD, fdD = lambdifyFormula(D,"D")

# Define values and their errors
vL, vdL = mu1,sig1
vW, vdW = mu2,sig2

# Numerically evaluate expressions and print 
vD = fD(vL,vW)
vdD = fdD(vL,vdL,vW,vdW)


#Adding correlations (and also derivation, printing and lambdifying)
sigCorr = symbols("sigma_LW")
rho = symbols("rho_LW")

dD, fdD = myDiffWithCorr(D, "D", True)

sCorr = sqrt(rho*dL*dW)
fSC = lambdify((rho,dL,dW),sCorr)

vSigmaCorr = fSC(rho12,vdL,vdW)

# Numerically evaluate expressions and print 
vdD = fdD(vL,vdL,vW,vdW,vSigmaCorr)

lprint(fr'D = ({vD:.3f} \pm {vdD:.4f})\,\mathrm{{m}}')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>